In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# **Import Libs**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# **Introduction**

При выполнении EDA я хочу поэтапно решить такие проблемы : 
* Переполнение одинаковыми и пропущенными данными 
* Категориальные столбцы
* Пропущенные данные в столбцах
* Высокая корреляция данных 

# **Load Data**

In [ ]:
workDatasets = {
    "test": "/kaggle/input/home-credit-default-risk/application_test.csv",
    "train": "/kaggle/input/home-credit-default-risk/application_train.csv"
}

In [ ]:
train_data = pd.read_csv(workDatasets["train"], sep=',', header=0)
target = train_data['TARGET']
test_data = pd.read_csv(workDatasets["test"], sep=',', header=0)

# **Check Data**

In [ ]:
print(train_data.info(max_cols=122))
print(train_data.shape)

In [ ]:
print(test_data.info(max_cols=122))
print(test_data.shape)

**Cразу видно, что часть данных неполная и часть — категориальная, они отображаются как object. 
Поэтому надо будет их как то обработать.**

**Также поскольку колонок очень много, я постараюсь уменьшить их количество,чтоб можно было это нормально анализировать.**

# **Frequency distribution of data**

**Давайте посмотрим на частотное распределение данных и удалим те колонки, в которых более 90% строк занимают одинаковые значения.**

In [ ]:
drop_freq_features = []
for feature in train_data:
    if (train_data[feature].value_counts()*100/len(train_data[feature])).max() > 90 and feature not in ["TARGET", "NAME_CONTRACT_TYPE"]:
        drop_freq_features.append(feature)
print(len(drop_freq_features), " столбцов подлежат уничтожению.")
for feature in drop_freq_features:
    print(f"Feature : {feature}\nmoda :\n{(train_data[feature].value_counts()*100/len(train_data[feature])).max()}\n" + "-"*50)
train_data = train_data.drop(drop_freq_features, axis=1)
test_data = test_data.drop(drop_freq_features, axis=1)

**Таких оказалось 27 столбцов. Почти все они о том какую информацию человек предоставил,  и о совпадении мест жительства, регистрации и работы.**

# **Missing Values**

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = df.isnull().sum() * 100 / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing', 1 : '% of All'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of All', ascending=False).round(1)
        print (str(mis_val_table_ren_columns.shape[0]) +
              " столбцов с неполными данными.")
        return mis_val_table_ren_columns

In [ ]:
missing_values = missing_values_table(train_data)
missing_values.head(10)

**Видно, что присутствует 66 столбцов с неполными данными. Столбцы, в которых отсутствует >= 50% данных я решил удалить.**

In [ ]:
mis = missing_values[missing_values.iloc[:,1] >= 50].iloc[:,1]
drop_missing_values = []
for missing_value in mis.index:
    drop_missing_values.append(missing_value)
print(len(drop_missing_values), " столбец подлежит уничтожению")
for feature in drop_missing_values:
    print(f"Feature :\n{feature}\nMissing values % : {mis[feature]}")
train_data = train_data.drop(drop_missing_values, axis=1)
test_data = test_data.drop(drop_missing_values, axis=1)

In [ ]:
fill_missing_values = list(set(missing_values.index).difference(set(drop_missing_values)))
print(f"{len(fill_missing_values)} столбцов с пропущенными данными осталось.")
for feature in fill_missing_values:
    print(f"Feature : {feature}\ntype : {train_data[feature].dtype}\n", "-"*50)

**Поскольку 41 столбец удален, осталось решить проблему лишь с 25. Перед тем как что-то делать с ними, я разберусь с категориальными данными, потому что среди них тоже есть столбцы с пропущенными данными**

# **Catecorical Data**

In [ ]:
categorical_columns = train_data.select_dtypes(include=[object]).apply(pd.Series.nunique, axis = 0)
print(categorical_columns)
print("Categorical columns count : ", len(categorical_columns))

**У нас есть 13 столбцов категориальных данных, из которых 5 с количеством уникальных категорий <= 3. Есть 2 варината того, что можно сделать с этими столбцами.**
1. Label Encoding
2. One-Hot-encoding
**Я воспользуюсь Label Encoding для столбцов, количеством уникальных категорий <= 3, а для остальных One-Hot-encoding**

**Но прежде заполним пропуски. Для этого посмотрим на наши категориальные столбцы поближе.**

In [ ]:
categorical_data = train_data[train_data.select_dtypes(include=[object]).columns]
for feature in categorical_data:
    print("Values %")
    print( 100 * train_data[feature].value_counts() / len(train_data[feature]))
    if feature in fill_missing_values:
        print("Missing values : ", missing_values.at[feature, '% of All'], "%")  
    train_data[feature].value_counts().plot(kind='bar')
    plt.show()

Пропущенные данные есть в столбцах : [OCCUPATION_TYPE, EMERGENCYSTATE_MODE, NAME_TYPE_SUITE].
* NAME_TYPE_SUITE
Здесь можно просто заполнить пропущенные значения Модой, поскольку missing % 0.4, а Мода составляет 80% от всех данных.
* EMERGENCYSTATE_MODE
Ее я решил удалить, потому что на 51.8% он заполнен одинаковыми данными и при этом 47.4% данных пропущено.
* OCCUPATION_TYPE
Заполним потом.

In [ ]:
train_data.drop(['EMERGENCYSTATE_MODE'], axis=1)
test_data.drop(['EMERGENCYSTATE_MODE'], axis=1)
mod = train_data['NAME_TYPE_SUITE'].mode()
train_data['NAME_TYPE_SUITE'] = train_data['NAME_TYPE_SUITE'].fillna(mod[0])
test_data['NAME_TYPE_SUITE'] = test_data['NAME_TYPE_SUITE'].fillna(mod[0])

In [ ]:
for feature in categorical_columns.index:
    if train_data[feature].nunique()  <= 3:
        train_data[feature] = train_data[feature].astype('category')
        train_data[feature] = train_data[feature].cat.codes
    if test_data[feature].nunique()  <= 3:
        test_data[feature] = test_data[feature].astype('category')
        test_data[feature] = test_data[feature].cat.codes
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [ ]:
print('Train data : ', train_data.shape)
print('Test data : ', test_data.shape)

**Так как количество вариантов в столбцах выборок не равное, количество столбцов теперь не совпадает. Требуется выравнивание — нужно убрать из тренировочной выборки столбцы, которых нет в тестовой.**

In [ ]:
train_data, test_data = train_data.align(test_data, join = 'inner', axis = 1)

print('Train data : ', train_data.shape)
print('Test data : ', test_data.shape)

train_data['TARGET'] = target

In [ ]:
missing_values_rest = missing_values_table(train_data)
print(missing_values_rest)

**Осталось лишь 22 столбца с неполными данными, а не 25, это можно обьяснить тем что было удалено 3 столбца во время выравнивания train_data к test_data.**

**Заполним их медианой.**

In [ ]:
for feature in missing_values_rest.index:
    mean = train_data[feature].mean()
    train_data[feature] = train_data[feature].fillna(mean)
    test_data[feature] = test_data[feature].fillna(mean)

# **Correlation Data**

In [ ]:
corrMatt = train_data.corr()
correlations = corrMatt['TARGET'].sort_values()
print('Наивысшая позитивная корреляция: \n', correlations.tail(15))
print('\nНаивысшая негативная корреляция: \n', correlations.head(15))

**Таким образом, все данные слабо коррелируют с TARGET. Выделяется EXT_SOURCE_2 EXT_SOURCE_3 и DAYS_BIRTH, но их корреляционный коэфициент все равно меньше 0.7 и больше -0.7, так что их можно и не трогать.**

In [ ]:
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, cmap="Greens", mask= mask, vmax=.8, square=True)

**Видно, что данные особо между собой не коррелируют. Так что на этом EDA можно завершить**

# **Model**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

if 'TARGET' in train_data:
    train = train_data.drop(labels = ['TARGET'], axis=1)
else:
    train = train_data.copy()

features = list(train.columns)
test = test_data.copy()

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Формат тренировочной выборки: ', train.shape)
print('Формат тестовой выборки: ', test.shape)

**Вначале мы очистили train_data от столбца TARGET. Далее мы нормализировали наши датасеты. И видим, что до начала учебы у нас осталось 158 столбцов.**

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C = 0.0001)

log_reg.fit(train, target)
LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


log_reg_pred = log_reg.predict_proba(test)[:, 1]

**Теперь модель можно использовать для предсказаний. Метод predict_proba даст на выходе массив m x 2, где m - количество наблюдений, первый столбец - вероятность 0, второй - вероятность 1. Нам нужен второй (вероятность невозврата).**

In [ ]:
submit = pd.DataFrame(log_reg_pred, columns = ['TARGET'])
submit['SK_ID_CURR'] = test_data['SK_ID_CURR']
submit.to_csv('submission.csv', index = False)

**Записали результат в submission.csv. Теперь можно сабмитить :)**

**Во время выполнения работы, я подумал, что стоило копировать датасет и уже в копиях удалять столбцы, а не в оригинале, потому что было пару раз когда хочу внести изменение вначале ноутбука, но не могу это сделать быстро, из-за того что нужно по новой запускать ноутбук. Но мне эта мысль пришла уже когда я почти все доделал и переделывать не был готов. Но в следующей работе планирую это применить. Так же стоило больше времени уделить корреляции столбов друг к другу, но их слишком много. Также заполнение пропущенных данных можно проводить учитывая корреляцию этих столбцов к столбцу TARGET, но опять все упиралось в их количество. А так работа получилась достаточно интересная.**